University of Michigan - ROB 101 Computational Linear Algebra

# Homework 4: Building Your Own Functions for Solving Equations
### Due: 10/08 at 9 PM Eastern

#### Purpose:
   Up to now, we have been wrting "scripts", that is, a series of commands that allow us to accomplish a goal. Scripts are are great tool and a super quick way to prototype ideas. We are now going to move on to writing "functions", which allow us to call our operations in a much easier manner. 

   Another thing we'll do is some error checking. Once you have hidden your code behind a function, someone else might try to use it in a manner that will throw bugs or much worse, produce erroneous results without throwing a bug and hence without warning the user. The funny part is, that "user" is often us! We build a function and then we forget exactly all of the special requirements on the data that the function uses to produce an answer! 
- Skills:
     - Writing functions
     - Using functions
     - Building in error checks
- Knowledge:
    - Learn to read error messages for detail
    - Confidence built in writing functions
   
#### Task:  
Read and execute the examples provided, then fill in code as directed to build your own functions, and then use them.

### Example: Forward Substitution

In [1]:
#computes the solution x to a system Lx = b where L is a lower triangular matrix and b is a column vector
#note the structure we have used: we being by declaring "function" and we end by declaring "end". On 
# the line before "end", we specify that value or values we are going to "return".
# Everything in between is operating on the data passed to the function.

function forwardsub(L, b)
    # START of our computations
    n = length(b)
    x = Vector{Float64}(undef, n); #initialize an x vector of the correct size
    x[1] = b[1]/L[1,1] #find the first entry of x
    for i = 2:n #find every entry from the 2nd to the end
        x[i]=(b[i]- (L[i,1:i-1])' *x[1:i-1] )./L[i,i] 
        #notice that we used a transpose operator to get the row of L
    end
    # END of our computations. 
    return x
end
#Everything between the function header and this last "end" statement
#is essentailly the same as a script. It's that easy to build a function!

forwardsub (generic function with 1 method)

In [5]:
#Here is an example where the function works!
L = [1 0 0 ; 2 3 0; 4 5 6]
b = [1; 8; 32]
forwardsub(L, b)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

### Problem 1.  Errors in the function


In [6]:
#Here is one way that we can break our function:
L = [6 0 0 ; 2 1 0; 8 5 0]
b = [3; 2; 3]
forwardsub(L, b)

3-element Vector{Float64}:
   0.5
   1.0
 -Inf

### a) Run the cell above, then explain, in a sentence, why the error occurred
Hint: It may help to write out the matrix L, then walk through the steps of the function

Answer:

The last row of L has only x1 and x2 , the x3 is zero so the system of equations only has one two unknowns. There is a division by zero in the last row while trying to find the x3 resulting in Infinity result.

In [7]:
#Here is another way that it could break:
L = [5 0 0 ; 1 4 0; 7 3 2]
b = [3; 2; 3; 4]
forwardsub(L, b)

LoadError: BoundsError: attempt to access 3×3 Matrix{Int64} at index [4, 1:3]

### b) Run the cell above and explain why the error occurred
Hint: Write the system in regular(non-matrix) notation.  We have done the first equation for you:

$$5x_1 + 0x_2 + 0x_3 = 3$$ 

Answer:

The dimentions are not appropriate. The B matrix is a column vector of with 4 elements while there are only 3 equations "columns" in the L matrix. This results in indexing of out of range enelmts in our algorithm.

In [8]:
#The above cases produced errors, so we knew to look for a problem in the data. 
#This example will not produce an error, but it also doesn't produce the solution we expect
using Random 
Random.seed!(4321) 
L=rand(3,3) 
b=rand(3,1) 
x=forwardsub(L, b)
@show L
@show x
@show b

println("Oops!")
solutionError=L*x-b

L = [0.1619496289112512 0.36326986232801595 0.6910169607353331; 0.1385032717390522 0.3700559311804976 0.08205602210008567; 0.6355055732000539 0.8332539255123794 0.6441883306325207]
x = [5.390839867286872, -0.11081974215841482, -4.355171929514309]
b = [0.8730445160270877; 0.7056394561629349; 0.5280168598206405]
Oops!


3×1 Matrix{Float64}:
 -3.0497551426899285
 -0.35736808409789894
  0.0

### c) Run the cell above and explain why x does not satisfy Lx = b
Hint:  You may want to add additional print statements in order to see the matrices you are using.

Answer:

The matrix L is nto a lower triangle matrix.

### Error Checking:  An example
Above, you experienced three possible ways that this function can fail.  Let's build a few error checks into the function so that we can tell what is going wrong.  Follow along with the comments in the code below to see how we can do these checks.

Now, if you go back and run the erroneous cells from above, the function will print out a more helpful error.

In [9]:
#I copied the forwardsub function from above
function forwardsub(L, b)
    # Now before we start our computations, let's make sure that there will be no errors
    n = length(b)
    
    #first, let's check that our inputs are the right size
    (rows, cols) = size(L)
    if rows != cols
        print("L is not a square")
        return 0
    end
    if rows != n
        print("L and b are not size compatible")
        return 0
    end
    #if we got to here, that means we have a matrix and vector of compatible sizes
    
    #now, let's check that L is actually lower triangular, eg: every element above the diagonal is zero
    #also check to make sure that there are no zeros on the diagonal
    for i in 1:n
        for j in 1:n
            if j > i
                if L[i, j] != 0
                    print("L is not lower triangular")
                    return 0
                end
            elseif j == i
                if L[i, j] == 0
                    print("There is a zero on the diagonal")
                    return 0
                end
            end
        end
    end
                    
    
    #Now we can actually do the forward substitution, now that we found the arguments to be acceptable
    
    x = Vector{Float64}(undef, n); #initialize an x vector of the correct size
    x[1] = b[1]/L[1,1] #find the first entry of x
    for i = 2:n #find every entry from the 2nd to the end
        x[i, 1] = (b[i]- (L[i,1:i-1])' *x[1:i-1] )./L[i,i] 
        #notice that we used a transpose(') operator to get the row of L
    end
    # END of our computations. Everything between START and END
    # is essentailly the same as a script. It's that easy to build a function!
    return x
end

forwardsub (generic function with 1 method)

### Problem 2.  Write a function for back substitution.
Use the function given to you in problem 1, and section 3.5 of the ROB 101 booklet to help you construct your answer.  Think about just doing forward substitution, but backwards.  

Hint:  In order to make a for loop that goes backwards, you can either use

**for i in reverse(1:n)**

or

**for i in n:-1:1**  (where the second parameter(-1) is the step size)

In [10]:
#computes the solution x to a system Ux = b where U is a lower triangular matrix and b is a column vector
function backwardsub(U, b)
    n = length(b)
    x = Vector{Float64}(undef, n) #initialize an x vector of the correct size
    
    
    #fill in the rest of the function here.  You should think about what the forward substitution algorithm looked like
    # your code here
    x[n] = b[n]/U[n,n] #find the last entry of x
    for i in n-1:-1:1
        x[i]=(b[i]- (U[i,i:n])' *x[i:n] )./U[i,i] 
    end
    
    return x
    
end
    

backwardsub (generic function with 1 method)

In [11]:
#Unit test:  Your answer should be [1, 2, 3] --a column vector
#Not getting what you expect?  Check to see if you are using the correct variables (L vs U)
U = [4 5 6; 0 2 3; 0 0 1 ]
b = [32; 13; 3]
answer2a = backwardsub(U, b)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

### Problem 3.  Add Error testing, like the example from Problem 1, to your function for backwardsub
Copy your code from problem 2, and make sure that you are checking for these errors:
* Check that the matrix U is a square and is compatible with the size of b
* Check that the matrix U is upper triangular
* Check that there are no zeros on the diagonal of U

**If you hit an error, print a relevant error message and return 0 to exit the function**

In [12]:
#copy your code from above, and add error checks
#Hint:  Look at the error checking example for the forwardsub function

# your code here
function backwardsub(U, b)
    # Now before we start our computations, let's make sure that there will be no errors
    n = length(b)
    
    #first, let's check that our inputs are the right size
    (rows, cols) = size(U)
    if rows != cols
        print("U is not a square")
        return 0
    end
    if rows != n
        print("U and b are not size compatible")
        return 0
    end
    #if we got to here, that means we have a matrix and vector of compatible sizes
    
    #now, let's check that U is actually Upper triangular, eg: every element below the diagonal is zero
    #also check to make sure that there are no zeros on the diagonal
    for i in 1:n
        for j in 1:n
            if i > j
                if U[i, j] != 0
                    print("U is not Upper triangular")
                    return 0
                end
            elseif j == i
                if U[i, j] == 0
                    print("There is a zero on the diagonal")
                    return 0
                end
            end
        end
    end
                    
    
    #Now we can actually do the forward substitution, now that we found the arguments to be acceptable
    
    x = Vector{Float64}(undef, n); #initialize an x vector of the correct size
    x[n] = b[n]/U[n,n] #find the last entry of x
    for i in n-1:-1:1
        x[i]=(b[i]- (U[i,i:n])' *x[i:n] )./U[i,i] 
    end
    # END of our computations. Everything between START and END
    # is essentailly the same as a script. It's that easy to build a function!
    return x
end

backwardsub (generic function with 1 method)

In [13]:
#The function should work here.  There should not be any error message
U1 = [1 2 3; 0 4 5; 0 0 6]
b1 = [14; 23; 18]
backwardsub(U1, b1)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [14]:
#You should have an error message about a zero on a diagonal
U2 = [6 8 1 ; 0 0 3; 0 0 4]
b2 = [3; 2; 3]
backwardsub(U2, b2)

There is a zero on the diagonal

0

In [15]:
#You should have an error message that U is not upper triangular
U3 = [2 8 1 9; 3 0 3 9; 0 0 4 2; 0 0 0 1]
b3 = [3; 2; 3; 1]
backwardsub(U3, b3)

U is not Upper triangular

0

In [16]:
#These matrices are not size compatible, your error should reflect that
U4 = [5 4 1; 6 7 2; -7 2 1]
b4 = [5; 1; 2; 6]
backwardsub(U4, b4)

U and b are not size compatible

0

### Problem 4.  LU decomposition with Permutations
Julia has a built-in LU Factorization function. Below is an example of how to access the properties of the lu() function.  

In [17]:
using LinearAlgebra
M = [0 4 2; 10 2 1; 1 1 1 ]
# F is a variable that holds all the data from the factorization. It contains all three matrices, L,U, and P
F = lu(M)
L =  F.L #lower triangular factor
@show L
U =  F.U #upper triangular factor
@show U
P = F.P #Permutation matrix
@show P
##properties of the factorization and a check that we know how to use it!
Check = P*M-L*U
#this should be a matrix of zeros if PM = LU

L = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.1 0.2 1.0]
U = [10.0 2.0 1.0; 0.0 4.0 2.0; 0.0 0.0 0.5]
P = [0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 0.0 1.0]


3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

### a) Return the Lower triangular matrix of the LU decomposition of matrix G
$$ G = \begin{bmatrix}6&7&2&9\\1&-3&-5&6\\-8&2&-3&-4\\0&0&2&1\end{bmatrix} $$

In [18]:
#Return your answer as a variable named answer4a
#replace the lines below with your code

using LinearAlgebra
G = [6  7  2  9; 
     1 -3 -5  6;
    -8  2 -3 -4;
     0  0  2  1]
# F is a variable that holds all the data from the factorization. It contains all three matrices, L,U, and P
F = lu(G)
L =  F.L #lower triangular factor
@show L
U =  F.U #upper triangular factor
@show U
P = F.P #Permutation matrix
@show P
##properties of the factorization and a check that we know how to use it!
@show Check = P*G-L*U;
#this should be a matrix of zeros if PM = LU

answer4a = L

L = [1.0 0.0 0.0 0.0; -0.75 1.0 0.0 0.0; -0.125 -0.3235294117647059 1.0 0.0; -0.0 0.0 -0.3665768194070081 1.0]
U = [-8.0 2.0 -3.0 -4.0; 0.0 8.5 -0.25 6.0; 0.0 0.0 -5.455882352941177 7.4411764705882355; 0.0 0.0 0.0 3.7277628032345014]
P = [0.0 0.0 1.0 0.0; 1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0]
Check = P * G - L * U = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


4×4 Matrix{Float64}:
  1.0     0.0        0.0       0.0
 -0.75    1.0        0.0       0.0
 -0.125  -0.323529   1.0       0.0
 -0.0     0.0       -0.366577  1.0

In [19]:
#autograder cell

### b) Return the Permutation matrix and the Upper triangular matrix of matrix C
$$ C = \begin{bmatrix}6&-2&8&-7&1\\-2&0&3&5&9\\9&-4&5&-1&0\\-8&2&-2&3&1\\6&0&-5&1&9\end{bmatrix}$$

In [20]:
#return the permutation matrix as perm4b and the upper triangular matrix as upper4b

using LinearAlgebra
C = [6 -2  8 -7 1; 
    -2  0  3  5 9;
     9 -4  5 -1 0;
    -8  2 -2  3 1;
     6  0 -5  1 9]

# F is a variable that holds all the data from the factorization. It contains all three matrices, L,U, and P
F = lu(C)
L =  F.L #lower triangular factor
@show L
U =  F.U #upper triangular factor
@show U
P = F.P #Permutation matrix
@show P
##properties of the factorization and a check that we know how to use it!
@show Check = P*C-L*U;
#this should be a matrix of zeros if PM = LU

perm4b = P;
upper4b = U;


L = [1.0 0.0 0.0 0.0 0.0; 0.6666666666666666 1.0 0.0 0.0 0.0; 0.6666666666666666 0.24999999999999994 1.0 0.0 0.0; -0.2222222222222222 -0.3333333333333333 0.19753086419753094 1.0 0.0; -0.8888888888888888 -0.5833333333333333 -0.35802469135802445 0.10000000000000017 1.0]
U = [9.0 -4.0 5.0 -1.0 0.0; 0.0 2.6666666666666665 -8.333333333333332 1.6666666666666665 9.0; 0.0 0.0 6.75 -6.75 -1.2499999999999996; 0.0 0.0 0.0 6.666666666666667 12.246913580246915; 0.0 0.0 0.0 0.0 4.577777777777775]
P = [0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0]
Check = P * C - L * U = [0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -8.881784197001252e-16 1.1102230246251565e-16 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -1.7763568394002505e-15; 0.0 0.0 0.0 0.0 0.0]


In [21]:
#autograder cell

### Problem 5.  Use LU decompostition and forward/back substitution to solve the system of equations for x.
$$\begin{bmatrix}8&-18&-16&2 &0& -8& 18& 8& 12 &-20\\ -36& 41& 142 &21& -20 &6 -111& -106& -24& 190\\
    36 &-117 &-13& 38 &-25 &-60 &63& -31& 79 &9\\ 32& -104& 32& 44& 26& -114 &-82& 52& 92& -130\\
    24& -78& -10 &26& -46 &-30 &36 &-10& 70& -15\\ 12 &-19 &-70 &-43 &-48 &142& 190 &-7 &38 &64\\
    16& -52& -40& 26& -136& 80& 126& 1& 69 &39\\ 20 &-9 &-63 &-66& 190& -129& 49& -10& -129& -199\\
    -28& 51 &41 &-4 &-3 &22 &119 &-261 &-143 &383\\ 36 &-61 &-123 &10 &-88 &-55 &1 &113 &95 &-173\end{bmatrix} x = \begin{bmatrix}-1\\2\\-2\\8\\-9\\-7\\-7\\-2\\-1\\-5\end{bmatrix} $$

Hint: For a system $Ax = b$, if the LU decompositon without permutations is performed on A, then $LUx = b$, $Ux = y$, and $Ly = b$.  See page 49 of the ROB 101 course booklet for an example.

In [22]:
#Solve the problem with julia's lu() function and your forward and back substitution functions 
#Matrices A and b are declared for you
A = [8 -18 -16 2 0 -8 18 8 12 -20; -36 41 142 21 -20 6 -111 -106 -24 190;
    36 -117 -13 38 -25 -60 63 -31 79 9; 32 -104 32 44 26 -114 -82 52 92 -130;
    24 -78 -10 26 -46 -30 36 -10 70 -15; 12 -19 -70 -43 -48 142 190 -7 38 64;
    16 -52 -40 26 -136 80 126 1 69 39; 20 -9 -63 -66 190 -129 49 -10 -129 -199;
    -28 51 41 -4 -3 22 119 -261 -143 383; 36 -61 -123 10 -88 -55 1 113 95 -173]
b = [-1;2; -2; 8; -9; -7; -7; -2; -1; -5]

10-element Vector{Int64}:
 -1
  2
 -2
  8
 -9
 -7
 -7
 -2
 -1
 -5

In [23]:
using LinearAlgebra
#Remember to use lu(M, Val(false)) in order to prevent permutations

# F is a variable that holds all the data from the factorization. It contains all three matrices, L,U, and P
L, U = lu(A, Val(false))
@show L
@show U
##properties of the factorization and a check that we know how to use it!
@show CheckLU = A-L*U;
#this should be a matrix of zeros if PM = LU


y = forwardsub(L, b);
x = backwardsub(U, y);

@show x

@show isCorrect = isapprox(A*x, b);


L = [1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -4.5 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 4.5 0.9 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 4.0 0.8 -10.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 3.0 0.6000000000000001 1.0000000000000018 -2.220446049250313e-16 1.0 0.0 0.0 0.0 0.0 0.0; 1.5 -0.2 8.0 -1.75 1.6666666666666672 1.0 0.0 0.0 0.0 0.0; 2.0 0.4 9.0 -0.25 2.6666666666666674 0.8333333333333334 1.0 0.0 0.0 0.0; 2.5 -0.9 -10.0 -0.75 -3.000000000000001 -1.6666666666666665 5.999999999999982 1.0 0.0 0.0; -3.5 0.30000000000000004 9.0 -0.75 -1.6666666666666672 -0.5 7.999999999999974 -0.6666666666666659 1.0 0.0; 4.5 -0.5 4.0 0.25 2.333333333333334 -0.9999999999999996 -6.999999999999972 -1.1842378929334998e-15 0.624999999999992 1.0]
U = [8.0 -18.0 -16.0 2.0 0.0 -8.0 18.0 8.0 12.0 -20.0; 0.0 -40.0 70.0 30.0 -20.0 -30.0 -30.0 -70.0 30.0 100.0; 0.0 0.0 -4.0 2.0 -7.0 3.0 9.0 -4.0 -2.0 9.0; 0.0 0.0 0.0 32.0 -28.0 -28.0 -40.0 36.0 0.0 -40.0; 0.0 0.0 0.0 0.0 -26.999999999999993 8.999999999999993 -9.000000000000021 12.000

In [24]:
#autograder cell